# VAE - 변이형 오토인코더

인코더 - 네트워크는 고차원 입력 데이터를 저차원 표현 벡터로 압축

디코더 - 네트워크는 주어진 표현 벡터를 원본 차원으로 다시 압축 해제



In [2]:
import tensorflow as tf

In [4]:
from tensorflow.keras import models

In [5]:
from tensorflow.keras.layers import Conv2D

In [10]:
from tensorflow.keras.layers import LeakyReLU

In [11]:
from tensorflow.keras.layers import Input

In [12]:
from tensorflow.keras.layers import backend

ImportError: cannot import name 'backend' from 'tensorflow.keras.layers' (C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\api\_v2\keras\layers\__init__.py)

In [ ]:
# 교재 Autoencoder

class Autoencoder():
    def __init__(self
        , input_dim
        , encoder_conv_filters
        , encoder_conv_kernel_size
        , encoder_conv_strides
        , decoder_conv_t_filters
        , decoder_conv_t_kernel_size
        , decoder_conv_t_strides
        , z_dim
        , use_batch_norm = False
        , use_dropout = False
        ):

        self.name = 'autoencoder'

        self.input_dim = input_dim
        self.encoder_conv_filters = encoder_conv_filters
        self.encoder_conv_kernel_size = encoder_conv_kernel_size
        self.encoder_conv_strides = encoder_conv_strides
        self.decoder_conv_t_filters = decoder_conv_t_filters
        self.decoder_conv_t_kernel_size = decoder_conv_t_kernel_size
        self.decoder_conv_t_strides = decoder_conv_t_strides
        self.z_dim = z_dim

        self.use_batch_norm = use_batch_norm
        self.use_dropout = use_dropout

        self.n_layers_encoder = len(encoder_conv_filters)
        self.n_layers_decoder = len(decoder_conv_t_filters)

        self._build()
        
        
        
    def _build(self):

        ### THE ENCODER
        encoder_input = Input(shape=self.input_dim, name='encoder_input')

        x = encoder_input

        for i in range(self.n_layers_encoder):
            conv_layer = Conv2D(
                filters = self.encoder_conv_filters[i]
                , kernel_size = self.encoder_conv_kernel_size[i]
                , strides = self.encoder_conv_strides[i]
                , padding = 'same'
                , name = 'encoder_conv_' + str(i)
                )

            x = conv_layer(x)

            x = LeakyReLU()(x)

            if self.use_batch_norm:
                x = BatchNormalization()(x)

            if self.use_dropout:
                x = Dropout(rate = 0.25)(x)

        shape_before_flattening = K.int_shape(x)[1:]

        x = Flatten()(x)
        encoder_output= Dense(self.z_dim, name='encoder_output')(x)

        self.encoder = Model(encoder_input, encoder_output)

In [ ]:
class encoder() :
    def __init__(self
            , input_dim  # input  값의 형태
            , encoder_conv_filters  # 인코더의 convolution layer에서 사용할 필터의 사이즈 
            , encoder_conv_kernel_size # 인코더 convolution layer의 커널 사이즈
            , encoder_conv_strides,
                 
            , decoder_conv_t_filters # 디코더의 필터
            , decoder_conv_t_kernel_size # 디코더 커널 사이즈
            , decoder_conv_t_strides # 디코너 stride
                 
            , z_dim # z의 dim???? 무슨 말이죠
            , use_batch_norm = False # Batch Normalzatino 디폴트는 안쓰는걸로 하자
            , use_dropout = False # dropout - 과적합 방지를 위해서, 이전 층의 유닛을 랜덤하게 0 으로 만들어버림
            ) :
        self.name = 'autoencoder'

        self.input_dim = input_dim
        self.encoder_conv_filters = encoder_conv_filters
        self.encoder_conv_kernel_size = encoder_conv_kernel_size
        self.encoder_conv_strides = encoder_conv_strides
        self.decoder_conv_t_filters = decoder_conv_t_filters
        self.decoder_conv_t_kernel_size = decoder_conv_t_kernel_size
        self.decoder_conv_t_strides = decoder_conv_t_strides
        self.z_dim = z_dim

        self.use_batch_norm = use_batch_norm
        self.use_dropout = use_dropout

        self.n_layers_encoder = len(encoder_conv_filters)
        
        self._build()
        # 일단 만들고 build를 def 단계에서 실행할 껀데 이제 build를 만들어보자
        
    def _build(self):
        
        # Encoder 부터 만들어봅시당
        encoder_input = Input(shape =self.input_dim, name = 'encoder_input')
        
        x = encoder_input
        
        for i in range(self.n_layers_encoder) :# n_layers_encoder 는 encode_conv_filter 의 길이 즉, 여기선 4개의 층
            conv_layer = Conv2D(filters = self.encoder_conv_filters[i],
                               kernel_size = self.encoder_conv_kernel_size[i],
                               strides= self.encoder_conv_strides[i],
                               padding = 'same',
                               name = 'encoder_conv_'+str(i)
                               )
            x = conv_layer(x)
            x = LeakyReLU()(x)
        
        #shape_before_flattening = K.int_shape(x)[1:]
        # tf 도 같이 사용하고 있으므로 굳이 keras backend 함수 안가져오고, tensorflow 로 처리
        shape_before_flattening = tf.shape(x).numpy()[1:]
        
        x = Flatten()(x)
        
        encoder_output = Dense(self.z_dim, name= 'encoder_output')(x)
        
        # 인코더를 하나의 Model로 만드는 건가
        # input과 output 그리고 그걸 계산하는 모든 레이어를 포함한다.
        # 미친 기능인거 같다
        self.encoder = Model(encoder_input, encoder_output)
        
        

## 인코더

### input layer
### conv + leaky relu * 4
### Flatten
### Dense

In [ ]:
encoder_input = Input()